# Session 6 — Modules and libraries, Exceptions, Files, Excel, Pandas, and SQLite

This notebook covers:
- Modules vs packages vs libraries
- Exploring built-in modules (`math`, `random`, `datetime`, `os`, `pathlib`)
- Creating and importing a custom module (`mymath.py`)
- Error & exception handling (`try/except/else/finally`)
- Text file operations (`read`, `write`, `append`, `.strip()`)
- Working with `.xlsx` files
- Pandas basics: `DataFrame`, read/write Excel
- SQLite database connectivity: `sqlite3`, `.execute()`, `fetchone/fetchall`, `commit`



## 1) Modules vs Packages vs Libraries

- **Module**: usually a single `.py` file containing code.
  - Example: `math`, `random`.
- **Package**: a folder containing modules (often includes `__init__.py`).
  - Example: `pandas` (package) with many submodules.
- **Library**: informal term meaning “collection of modules/packages” used for some purpose.

### Import styles
- `import module`
- `import module as alias`
- `from module import name`
- `from module import name as alias`


In [ ]:
# Different import styles
import math
import random as rnd
from datetime import datetime

print("sqrt(25) =", math.sqrt(25))
print("random int 1..10 =", rnd.randint(1, 10))
print("now =", datetime.now())


## 2) Exploring Built-in Modules

Python’s **standard library** includes many built-in modules.

Two super useful helpers:
- `dir(module)` → list names inside the module
- `help(module_or_function)` → docs


In [ ]:
import math

print("Some names in math:")
print(dir(math)[:15], "...")

print("\nHelp on math.sqrt:")
help(math.sqrt)


### More built-in modules (examples)
- `os` and `pathlib` help with files and folders
- `sys` gives Python runtime info

Below we’ll check current folder and create a path safely.

In [ ]:
import os
from pathlib import Path

print("Current working directory:")
print(os.getcwd())

p = Path("data") / "example.txt"  # safe cross-platform path building
print("Example path:", p)


## 3) Custom Module Example (`mymath.py`)

A custom module is just a `.py` file you create.

### If you are using Jupyter Notebook:
You can create a file from a cell using `%%writefile mymath.py`.

Then you import it like any other module.

> Note: If you modify the file after importing, you may need to restart the kernel or use `importlib.reload`.


In [ ]:
%%writefile mymath.py
# mymath.py — custom module example

def add(a, b):
    return a + b

def is_even(n):
    return n % 2 == 0

def area_circle(r):
    import math
    return math.pi * r * r


In [ ]:
import mymath

print(mymath.add(3, 4))
print(mymath.is_even(10))
print(round(mymath.area_circle(2), 4))


### Reloading a module (when you edit it)
If you change `mymath.py` and want to re-import without restarting:

```python
import importlib
importlib.reload(mymath)
```


## 4) Errors and Exception Handling

### Why exceptions?
Exceptions let your program handle problems *gracefully* instead of crashing.

Common exceptions:
- `ValueError` (bad conversion like `int('abc')`)
- `ZeroDivisionError`
- `FileNotFoundError`
- `TypeError`

### Main patterns
- `try/except`
- `try/except/else`
- `try/except/finally`
- `try/except/else/finally`


In [ ]:
# try/except with multiple exception types

try:
    x = int(input("Enter an integer: "))
    print("10 / x =", 10 / x)
except ValueError:
    print(" You must enter a valid integer.")
except ZeroDivisionError:
    print(" You cannot divide by zero.")


In [ ]:
# try/except/else/finally

try:
    x = int(input("Enter an integer again: "))
except ValueError:
    print("Invalid input")
else:
    # runs only if no exception
    print("You entered:", x)
finally:
    # always runs
    print("(finally) This runs whether there was an error or not")


## 5) Text File Operations

### File modes
- `'r'` read (file must exist)
- `'w'` write (overwrites / creates)
- `'a'` append (adds to end / creates)

### `.strip()`
Removes whitespace from both ends (spaces, tabs, newlines). Great when reading lines.


In [ ]:
# Create a sample text file using write mode (overwrites)
with open("notes.txt", "w", encoding="utf-8") as f:
    f.write("  line one  \n")
    f.write("line two\n")
    f.write("   line three\n\n")

print("Created notes.txt")


In [ ]:
# Read the file and use strip() to clean lines
with open("notes.txt", "r", encoding="utf-8") as f:
    for line in f:
        cleaned = line.strip()
        if cleaned:  # skip empty lines after stripping
            print(cleaned)


In [ ]:
# Append mode adds content without overwriting
with open("notes.txt", "a", encoding="utf-8") as f:
    f.write("appended line\n")

print("Appended to notes.txt")


## 6) Working with Excel `.xlsx`

Two common ways:
1) **Pandas** (best for analysis) — `read_excel`, `to_excel`
2) **openpyxl** (best for cell-level editing/formatting)

> If you get an engine error, install: `pip install openpyxl`


## 7) Pandas Basics — DataFrame

A **DataFrame** is a table (rows + columns), like Excel/SQL.

We'll:
- Create a DataFrame
- Save it to Excel
- Read it back
- Filter + sort


In [ ]:
import pandas as pd

df = pd.DataFrame({
    "name": ["Ritesh", "Nikhil", "Phenome", "Rahul"],
    "score": [95, 88, 91, 77]
})

df


In [ ]:
# Write to Excel
df.to_excel("students.xlsx", index=False)
print("Wrote students.xlsx")


In [ ]:
# Read from Excel
df2 = pd.read_excel("students.xlsx")
print(df2.head())


In [ ]:
# Filter and sort
top = df2[df2["score"] >= 90].sort_values("score", ascending=False)
top


In [ ]:
# Export filtered results
top.to_excel("top_students.xlsx", index=False)
print("Wrote top_students.xlsx")


## 8) SQLite Database Connectivity (`sqlite3`)

SQLite stores your database in a **single file** (e.g., `school.db`).

Core steps:
1) `sqlite3.connect(...)`
2) `conn.cursor()`
3) `cursor.execute(SQL, params)`
4) `fetchone()` / `fetchall()` for SELECT
5) `conn.commit()` for INSERT/UPDATE/DELETE
6) `conn.close()`

### Important
Use **parameterized queries** with `?` — safer and avoids quoting bugs.


In [ ]:
import sqlite3

# Create/connect to DB
conn = sqlite3.connect("school.db")
cur = conn.cursor()

# Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    score INTEGER NOT NULL
)
""")

conn.commit()
conn.close()

print("Created/verified school.db and students table")


In [ ]:
import sqlite3

with sqlite3.connect("school.db") as conn:
    cur = conn.cursor()

    # Insert rows (parameterized)
    cur.execute("INSERT INTO students (name, score) VALUES (?, ?)", ("Ritesh", 95))
    cur.execute("INSERT INTO students (name, score) VALUES (?, ?)", ("Nikhil", 88))
    cur.execute("INSERT INTO students (name, score) VALUES (?, ?)", ("Phenome", 91))
    cur.execute("INSERT INTO students (name, score) VALUES (?, ?)", ("Rahul", 77))

print("Inserted sample rows")


In [ ]:
import sqlite3

with sqlite3.connect("school.db") as conn:
    cur = conn.cursor()

    # SELECT + fetchall
    cur.execute("SELECT name, score FROM students WHERE score >= ? ORDER BY score DESC", (90,))
    rows = cur.fetchall()

print("Students with score >= 90:")
for name, score in rows:
    print(name, score)


In [ ]:
import sqlite3

with sqlite3.connect("school.db") as conn:
    cur = conn.cursor()

    # UPDATE
    cur.execute("UPDATE students SET score = ? WHERE name = ?", (99, "Ritesh"))

    # DELETE
    cur.execute("DELETE FROM students WHERE name = ?", ("Rahul",))

print("Updated Ruchik, deleted Swapna")


In [ ]:
import sqlite3

with sqlite3.connect("school.db") as conn:
    cur = conn.cursor()
    cur.execute("SELECT id, name, score FROM students ORDER BY id")
    all_rows = cur.fetchall()

print("All rows now:")
for r in all_rows:
    print(r)
